## <span style='color:#ff5f27'> 📝 Imports

In [1]:
# !pip install hopsworks==3.7.0 --quiet
# !pip install mimesis==15.1.0 --quiet
# !mkdir -p features
# !cd features && wget https://raw.githubusercontent.com/jimdowling/pydata-tiktok/main/features/users.py
# !cd features && wget https://raw.githubusercontent.com/jimdowling/pydata-tiktok/main/features/videos.py
# !cd features && wget https://raw.githubusercontent.com/jimdowling/pydata-tiktok/main/features/interactions.py

In [1]:
import pandas as pd
import numpy as np
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from features.users import generate_users
from features.videos import generate_video_content
from features.interactions import generate_interactions

## <span style='color:#ff5f27'> ⚙️ Data Generation</span>


### <span style='color:#ff5f27'> 👥 Users Data Generation</span>

In [2]:
user_data = generate_users(25_000)

data_users_df = pd.DataFrame(user_data)
data_users_df.head()

,user_id,gender,age,country
0,CI528F,Male,78,Togo
1,QF306P,Male,67,Tuvalu
2,GP017X,Other,70,Oman
3,EP696V,Other,44,Tristan da Cunha
4,JB392V,Male,54,Macau SAR China


### <span style='color:#ff5f27'> 🎥 Content Data Generation</span>


In [3]:
# Generate data for 25_000 videos
video_data = generate_video_content(25_000, historical=True)

data_video_df = pd.DataFrame(video_data)
data_video_df.head()

,video_id,category,views,likes,video_length,upload_date
0,6BV43W,Cooking,7384,2442,93,2023-01-08
1,4VU05G,Education,187248,185721,196,2023-03-31
2,0XU94E,Entertainment,243973,39414,50,2023-12-05
3,4NR30S,Education,44134,13673,104,2023-04-28
4,5SK99A,Technology,284122,82594,228,2024-01-05


### <span style='color:#ff5f27'> 🔗 Interactions Generation</span>


In [4]:
num_interactions = 250_000

# Generate interactions
interactions = generate_interactions(num_interactions, user_data, video_data)

data_interactions_df = pd.DataFrame(interactions)
data_interactions_df.head()

,interaction_id,user_id,video_id,interaction_type,watch_time
0,0033-33-3612,AH790A,3ZD26H,share,25
1,4003-03-9300,DN586P,9UD49Q,skip,118
2,2210-96-8439,AL581H,0HO34F,skip,51
3,1732-50-0971,RD647X,4LC20K,skip,171
4,4300-85-7482,EG272W,7ZA82N,skip,58


## <span style="color:#ff5f27">👮🏻‍♂️ Great Expectations </span>

In [5]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_users_df = ge.from_pandas(data_users_df)

# Initialize the expectation suite
expectation_suite_users = ge_users_df.get_expectation_suite()
expectation_suite_users.expectation_suite_name = "user_data_suite"

# Expectation: Age should be between 0 and 120
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "age", "min_value": 12, "max_value": 100}
    )
)

# Expectations: Columns should not have null values
for column in ge_users_df.columns:
    expectation_suite_users.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Gender should only contain specific values
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={"column": "gender", "value_set": ["Male", "Female", "Other"]}
    )
)

{"kwargs": {"column": "gender", "value_set": ["Male", "Female", "Other"]}, "meta": {}, "expectation_type": "expect_column_distinct_values_to_be_in_set"}

In [6]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_video_df = ge.from_pandas(data_video_df)

# Initialize the expectation suite
expectation_suite_videos = ge_video_df.get_expectation_suite()
expectation_suite_videos.expectation_suite_name = "video_data_suite"

# Expectation: Views, Likes, and Video Length should be non-negative
for column in ["views", "likes", "video_length"]:
    expectation_suite_videos.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={"column": column, "min_value": 0, "max_value": None}
        )
    )

# Expectation: Valid date format for upload_date
expectation_suite_videos.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "upload_date"}
    )
)

{"kwargs": {"column": "upload_date"}, "meta": {}, "expectation_type": "expect_column_values_to_be_dateutil_parseable"}

In [7]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_interactions_df = ge.from_pandas(data_interactions_df)

# Initialize the expectation suite
expectation_suite_interactions = ge_interactions_df.get_expectation_suite()
expectation_suite_interactions.expectation_suite_name = "interactions_data_suite"

# Expectations: Non-null values in all columns
for column in ge_interactions_df.columns:
    expectation_suite_interactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Validate interaction types
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "interaction_type",
            "value_set": ['like', 'dislike', 'view', 'comment', 'share', 'skip']
        }
    )
)

# Expectation: Positive watch time
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "watch_time", "min_value": 0}
    )
)

{"kwargs": {"column": "watch_time", "min_value": 0}, "meta": {}, "expectation_type": "expect_column_values_to_be_between"}

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [8]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>


In [9]:
users_fg = fs.get_or_create_feature_group(
    name="users",
    description="Users data.",
    version=1,
    primary_key=["user_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_users,
)

users_fg.insert(data_users_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/741723
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/398/fs/335/fg/741723


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████| Rows 25000/25000 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: users_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/users_1_offline_fg_materialization/executions
Done ✅


In [10]:
videos_fg = fs.get_or_create_feature_group(
    name="videos",
    description="Videos data.",
    version=1,
    primary_key=["video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_videos,
)

videos_fg.insert(data_video_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/740722
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/398/fs/335/fg/740722


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████| Rows 25000/25000 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: videos_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/videos_1_offline_fg_materialization/executions
Done ✅


In [11]:
interactions_fg = fs.get_or_create_feature_group(
    name="interactions",
    description="Interactions data.",
    version=1,
    primary_key=["interaction_id", "user_id", "video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_interactions,
)

interactions_fg.insert(data_interactions_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/740723
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/398/fs/335/fg/740723


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████| Rows 1000000/1000000 | Elapsed Time: 01:10 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/interactions_1_offline_fg_materialization/executions
Done ✅


## <span style="color:#ff5f27">🪄 Ranking Feature Group </span>


In [12]:
video_interactions_df = pd.merge(
    data_interactions_df, 
    data_video_df, 
    on='video_id', 
    how='inner',
)

ranking_df = pd.merge(
    video_interactions_df, 
    data_users_df, 
    on='user_id', 
    how='inner',
)

ranking_df['label'] = np.where(
    ranking_df.interaction_type.isin(['view', 'like', 'share', 'comment']), 
    1, 
    0,
)

ranking_df.drop(
    ['interaction_id', 'interaction_type', 'watch_time'], 
    axis=1, 
    inplace=True,
)

ranking_df.head()

,user_id,video_id,category,views,likes,video_length,upload_date,gender,age,country,label
0,AH790A,3ZD26H,Sports,39154,37332,25,2022-07-01,Male,47,Nepal,1
1,AH790A,0CT17D,Music,33929,1380,46,2023-06-17,Male,47,Nepal,1
2,AH790A,2QS27W,News,821,181,215,2022-12-13,Male,47,Nepal,0
3,AH790A,4IY92I,Entertainment,58921,2431,31,2023-04-29,Male,47,Nepal,0
4,AH790A,7YU96J,Music,283748,117396,49,2023-11-13,Male,47,Nepal,1


In [13]:
ranking_fg = fs.get_or_create_feature_group(
    name="ranking",
    description="Ranking Data.",
    version=1,
    primary_key=["user_id", "video_id"],
    online_enabled=True,
)

ranking_fg.insert(ranking_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/739716


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████| Rows 1002236/1002236 | Elapsed Time: 01:19 | Remaining Time: 00:00


Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/ranking_1_offline_fg_materialization/executions
Done ✅


---